# Set-Up Code

In [ ]:
OPENAI_API_KEY="*********"
OPENAI_PROJECT="*********"
OPENAI_ORG="*********"


In [2]:
# from utils.envs import OPENAI_API_KEY
import os
import base64
from typing import List, Dict
from openai import OpenAI
import openai
import glob
import pprint
import tqdm
import numpy as np
import json
import tiktoken
import pandas as pd
import re
import ast
import time
from datetime import datetime

### Initialize OpenAI Client

In [3]:
client = OpenAI(api_key=OPENAI_API_KEY, project=OPENAI_PROJECT, organization=OPENAI_ORG)

### Test that I can use Assistants & Responses API

In [4]:
# test that i can prompt a model

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! How can I use OpenAI's API to classify job descriptions?"}
    ]
)   

print(response.choices[0].message.content)

To use OpenAI's API to classify job descriptions, you can follow these general steps:

1. Sign up for access to OpenAI's API: Visit the OpenAI website and create an account to access their API services.

2. Retrieve API key: After signing up and being granted access, you will receive an API key that you can use to authenticate your requests.

3. Choose a suitable model: OpenAI provides various models that can be used for different natural language processing tasks. You need to select a model that is best suited for classifying job descriptions.

4. Build and send requests: Use the chosen model to build a request that includes the job description you want to classify. Send the request to OpenAI's API endpoint. Make sure to include your API key for authentication.

5. Receive and interpret the result: Once you send the request, you will receive a response containing the classification of the job description. Interpret the result to understand how the job description was classified.

6. I

In [5]:
response = client.responses.create(
    model="gpt-5",
    input = "Hello! How can I use OpenAI's API to classify job descriptions?",
    instructions = "You are a helpful assistant."
)

print(response.output[1].content[0].text)

Great use case. You’ve got a few good options depending on your goals (speed/cost vs. accuracy/consistency). Here’s a practical playbook with code.

1) Start with prompting + JSON schema (fastest to ship)
- When you have a known taxonomy (e.g., department, level, work mode, skills), ask the model to return only JSON that matches your schema. Set temperature=0 for consistency.

JavaScript (Node) using the Responses API:
- Use gpt-4.1-mini (cheap, strong for classification).
- response_format=json_schema with strict: true guarantees proper JSON.

const OpenAI = require("openai");
const client = new OpenAI();

const schema = {
  name: "JobClassification",
  schema: {
    type: "object",
    additionalProperties: false,
    properties: {
      department: {
        type: "string",
        enum: [
          "Software Engineering","Data Science","Data Engineering","Product Management",
          "Design","Marketing","Sales","Customer Support","Operations","Finance",
          "HR","IT","Secu

In [6]:
client.vector_stores.list()

SyncCursorPage[VectorStore](data=[VectorStore(id='vs_69025034e2fc81918fdcad91301db0cf', created_at=1761759285, file_counts=FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1), last_active_at=1761759310, metadata={}, name='column_mapper_v1', object='vector_store', status='completed', usage_bytes=9184, expires_after=None, expires_at=None, description=None), VectorStore(id='vs_68fdb1fa7e688191a6a4f690441695ea', created_at=1761456634, file_counts=FileCounts(cancelled=0, completed=5, failed=0, in_progress=0, total=5), last_active_at=1761456685, metadata={}, name='job_classifier_GPT_isha', object='vector_store', status='completed', usage_bytes=5282022, expires_after=None, expires_at=None, description=None), VectorStore(id='vs_68fba18f05408191853bfea00e219bc4', created_at=1761321359, file_counts=FileCounts(cancelled=0, completed=5, failed=0, in_progress=0, total=5), last_active_at=1761321615, metadata={}, name='job_classifier_gpt_no_aliases', object='vector_store', status='

# Read in Competitor Data
**1: Argenx\
2: KFH\
3: Mars\
4: WTW**

In [9]:
df_competitor_1 = pd.read_excel('./../Files/Competitor/Argenx_competitor.xlsx', sheet_name = 'WD report', header=None)
print(df_competitor_1.shape)
df_competitor_1.head(10)

(1687, 15)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,argenx Worker Overview - Total Compensation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Active Status,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Effective as of Date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(only car allowances),NaN,NaN,NaN,NaN
4,ID,Gender,Business Title,Country,argenx job level,Currency,Actual Annual Salary,Currency,Variable Pay - Plan,Target Variable Pay - Percent,Allowance - Car,Allowance - Phone,Allowance - Representation,Equity Amount - Annual,Cost Center - Name
5,100640,Male,"Head of Strategic Business Leaders, West",United States of America,Grade 44,USD,300489.75,USD,Field Incentive Plan,NaN,0,150,0,240000,Sales Force
6,101381,Male,"Head of Strategic Business Leaders, East",United States of America,Grade 44,USD,305767,USD,Field Incentive Plan,NaN,0,150,0,240000,Sales Force
7,103007,Male,"Head of Strategic Business Leaders, Central",United States of America,Grade 44,USD,334400,USD,Field Incentive Plan,NaN,1000,150,0,205000,Sales Force
8,100604,Female,"Strategic Business Leader, Atlanta Metro",United States of America,Grade 33,USD,275482.9,USD,Field Incentive Plan,NaN,0,150,0,105000,Sales Force
9,100605,Male,"Strategic Business Leader, South Texas",United States of America,Grade 33,USD,275482.9,USD,Field Incentive Plan,NaN,0,150,0,105000,Sales Force


# COMPETITOR FILE HEADERS EXTRACTION

## Use GPT-5-reasoning to extract a list of competitor header names

In [12]:
system_prompt_column_extraction = f"""You are a schema header extractor.

You will receive the TOP 10 ROWS of a rectangular table as a 2D array of strings, representing the first 10 rows of an Excel/CSV sheet read with header=None. True column headers might NOT be on row 1. Some headers may span MULTIPLE ROWS (hierarchical); merged Excel cells appear as blanks below the parent label. Some cells in the top 10 rows can already be DATA (not headers).

Your task:
- For each column (left→right), reconstruct its header and return a list of names with the SAME LENGTH as the number of columns.
- Reconstruct by:
  1) Forward-filling non-empty strings downward in that column (simulate merged cells).
  2) Identify header-ish lines vs data-ish lines: header-ish lines contain mostly words/symbols, data-ish lines are mostly numbers/dates/IDs. Stop including tokens once you hit data-ish content.
  3) Build the name by joining the remaining non-empty header tokens with a dot "." in top→down order (e.g., Country.India).
  4) Drop purely decorative tokens (e.g., "Table", "Report", company names, page titles).
  5) If a row looks like UNITS (%, USD, hrs, kg...), exclude it from the name.
  6) Fix obvious typos (e.g., "Conutry" -> "Country").

- Duplicates MUST be preserved: if the same final header appears multiple times, suffix occurrences in order as ".1", ".2", ".3", etc. (Example: EmployeeID, EmployeeID.1, EmployeeID.2)

- If a column has no usable header tokens, return "Unnamed_{{index}}" where index is the zero-based column index.

Output:
Return ONLY valid JSON in this exact schema and nothing else:
{{
  "headers": ["<col0_name>", "<col1_name>", "..."]
}}
The array length MUST equal the number of columns. No prose. No markdown. No trailing comments.
"""

In [13]:
def build_user_prompt_column_extraction(n_rows, df_10_rows):
 
 user_prompt = f"""Here is the table preview (top {n_rows} rows), read with header=None.

"grid_top10": {df_10_rows}

Please produce the JSON object as specified in the System Prompt. Remember:
- One name per header column in the DF.
- Use dot "." for hierarchy.
- Suffix duplicate names as .1, .2, ...
- Use Unnamed_{{index}} if needed.
"""
 
 return user_prompt

In [14]:
def construct_user_prompt(df, num_rows=10):

    # construct a string of the top 10 rows of DF
    sub = df.iloc[:num_rows].copy()
    sub = sub.where(sub.notna(), "")
    df_10_rows_str = [[str(x) for x in row] for row in sub.values.tolist()]

    # build user prompt
    user_prompt_column_extraction = build_user_prompt_column_extraction(num_rows, df_10_rows_str)

    return user_prompt_column_extraction


In [15]:
user_prompt_column_extraction_1 = construct_user_prompt(df_competitor_1)

In [16]:
# see system prompt
print(system_prompt_column_extraction)

You are a schema header extractor.

You will receive the TOP 10 ROWS of a rectangular table as a 2D array of strings, representing the first 10 rows of an Excel/CSV sheet read with header=None. True column headers might NOT be on row 1. Some headers may span MULTIPLE ROWS (hierarchical); merged Excel cells appear as blanks below the parent label. Some cells in the top 10 rows can already be DATA (not headers).

Your task:
- For each column (left→right), reconstruct its header and return a list of names with the SAME LENGTH as the number of columns.
- Reconstruct by:
  1) Forward-filling non-empty strings downward in that column (simulate merged cells).
  2) Identify header-ish lines vs data-ish lines: header-ish lines contain mostly words/symbols, data-ish lines are mostly numbers/dates/IDs. Stop including tokens once you hit data-ish content.
  3) Build the name by joining the remaining non-empty header tokens with a dot "." in top→down order (e.g., Country.India).
  4) Drop purely de

In [17]:
# see user prompt, competitor 1
print(user_prompt_column_extraction_1)

Here is the table preview (top 10 rows), read with header=None.

"grid_top10": [['argenx Worker Overview - Total Compensation', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['Active Status', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['Effective as of Date', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '(only car allowances)', '', '', '', ''], ['ID', 'Gender', 'Business Title', 'Country', 'argenx job level', 'Currency', 'Actual Annual Salary', 'Currency', 'Variable Pay - Plan', 'Target Variable Pay - Percent', 'Allowance - Car', 'Allowance - Phone', 'Allowance - Representation', 'Equity Amount - Annual', 'Cost Center - Name'], ['100640', 'Male', 'Head of Strategic Business Leaders, West', 'United States of America', 'Grade 44', 'USD', '300489.75', 'USD', 'Field Incentive Plan', '', '0', '150', '0', '240000', 'Sales Force'], ['101381', 'Male', 'Head of Strategic Business Leaders, East', 'United States of Amer

## Prompt GPT-5 with the competitor file data

In [18]:
def obtain_extracted_columns(user_prompt):
    start_time = time.time()
    response_competitor_columns = client.responses.create(input=user_prompt,
                                        instructions=system_prompt_column_extraction,
                                        model="gpt-5")
    end_time = time.time()
    elapsed_time = end_time - start_time
    extracted_columns = response_competitor_columns.output[-1].content[-1].text

    return extracted_columns, elapsed_time


In [19]:
competitor_1_columns, competitor_1_latency = obtain_extracted_columns(user_prompt_column_extraction_1)

print('latency:', competitor_1_latency)
print('extracted columns:', competitor_1_columns)

latency: 36.755266189575195
extracted columns: {
  "headers": ["ID", "Gender", "Business Title", "Country", "argenx job level", "Currency", "Actual Annual Salary", "Currency.1", "Variable Pay - Plan", "Target Variable Pay - Percent", "Allowance - Car", "Allowance - Phone", "Allowance - Representation", "Equity Amount - Annual", "Cost Center - Name"]
}


In [24]:
extracted_cols_json = json.loads(competitor_1_columns)
print('type:', type(extracted_cols_json))
print('# cols:', len(extracted_cols_json["headers"]))
print('EXTRACTED COLUMNS:')
print(extracted_cols_json["headers"])
print('type:', type(extracted_cols_json["headers"]))

type: <class 'dict'>
# cols: 15
EXTRACTED COLUMNS:
['ID', 'Gender', 'Business Title', 'Country', 'argenx job level', 'Currency', 'Actual Annual Salary', 'Currency.1', 'Variable Pay - Plan', 'Target Variable Pay - Percent', 'Allowance - Car', 'Allowance - Phone', 'Allowance - Representation', 'Equity Amount - Annual', 'Cost Center - Name']
type: <class 'list'>


In [30]:
# write to Responses/Extracted_Columns
with open("./../Responses/Extracted_Columns/Argenx_competitor_columns.txt", "w") as f:
    for item in extracted_cols_json["headers"]:
        f.write(f"{item}\n")

### Do this for the rest of the competitor files.

## COMPETITOR FILE 2

In [25]:
# let's see how Competitor Files 2-4 go

df_competitor_2 = pd.read_excel('./../Files/Competitor/KFH_competitor.xlsx', sheet_name = 'Employee Overview', header=None)
print(df_competitor_2.shape)
df_competitor_2.head(10)

(36190, 51)


,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1,17,19,20,21,22,23,35,36,...,111,NaN,NaN,123,NaN,NaN,NaN,140,142,143
3,NaN,Worker ID,Group Entity,Board level,Board level -1,Board level -2,Board level -3,Supervisory Org,Job Code,Job Profile Name,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Actual VP,Tenure,Annualized VP,Special recognition award (for those positions...,Sales Incentives,Profit Sharing,LTI,Lease Car Benefit Plan,Mobility Budget Amount (Year),Mobility Budget Discretionary (Year)
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,00006101,RGO,Retail NL (Carlo van Kemenade),Business Clients (Arjan Groen),BEDR MKB L&S (Stefan Lohuis),BEDR MKB Financieren (Joost Oorschot),BEDR MKB Service Fin. Midden-Oost (Chantal van...,NLJPFKO07RG001RG,NL Commercieel Ondersteuner D - Financieren,...,0,NaN,0,0,N,N,N,NaN,0,0
7,NaN,00007486,RGO,Retail NL (Carlo van Kemenade),Business Clients (Arjan Groen),BEDR MKB L&S (Stefan Lohuis),BEDR MKB Klantenservice (Saskia van Tilburg),BEDR MKB KS Rotterdam 3 (Jolanda Burggraaf),NLJPFKA05RG011RG,NL Medewerker All-finance B - KS MKB,...,0,NaN,0,0,N,N,N,NaN,0,0
8,NaN,00007535,RGO,Retail NL (Carlo van Kemenade),Insurance & Pensions (Irik van der Geld),I&P Regio Midden-Oost (Louis Miltenburg),I&P Regio MO Verzekeren Sector Agri (Fred Wisk...,I&P Regio MO Verzekeren Sector Agri (Fred Wisk...,NLJPFKO06RG000RG,NL Commercieel Ondersteuner C,...,0,NaN,0,0,N,N,N,NaN,0,0
9,NaN,00007540,RGO,Retail NL (Carlo van Kemenade),Operations (Bart Horsten),Operations Customer Journeys (Hilke Kersten),OPS Customer Journeys Nabestaandendesk (Paul v...,OPS Customer Journeys Nabestaanden team 3 (Eli...,NLJPFPT05RG006RG,NL Medewerker Klantondersteuning B - Beheer,...,0,NaN,0,0,N,N,N,NaN,0,0


In [26]:
user_prompt_column_extraction_2 = construct_user_prompt(df_competitor_2)
print(user_prompt_column_extraction_2)

Here is the table preview (top 10 rows), read with header=None.

"grid_top10": [['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'Use as fixed outside NL', '', 'Use as Fixed NL', '', '', '', '', '', '', '', '', '', '', ''], ['', '1', '17', '19', '20', '21', '22', '23', '35', '36', '37', '39', '', '6', '7', '8', '13', '41', '25', '27', '44', '46', '', '', '', '', '', '50', '55', '57', '59', '', '61', '91', '79+81+83+85', '109', '', '53', '107', '', '112', '111', '', '', '123', '', '', '', '140', '142', '143'], ['', 'Worker ID', 'Group Entity', 'Board level', 'Board level -1', 'Board level -2', 'Board level -3', 'Supervisory Org', 'Job Code', 'Job Profile Name', 'Position Title', 'Job Fa

In [27]:
competitor_2_columns, competitor_2_latency = obtain_extracted_columns(user_prompt_column_extraction_2)

print('latency:', competitor_2_latency)
print('extracted columns:', competitor_2_columns)

latency: 122.34330487251282
extracted columns: {
  "headers": [
    "Unnamed_0",
    "Worker ID",
    "Group Entity",
    "Board level",
    "Board level -1",
    "Board level -2",
    "Board level -3",
    "Supervisory Org",
    "Job Code",
    "Job Profile Name",
    "Position Title",
    "Job Family",
    "Area of Expertise",
    "Gender",
    "Date of Birth",
    "Age",
    "Continuous Service Date",
    "Comp Grade",
    "Country",
    "State",
    "Default Working Hours",
    "FTE Factor",
    "2024 Job Matching.KFH",
    "2024 Job Matching.McLagan",
    "2024 Job Matching.WTW",
    "2024 Job Matching.Discipline",
    "2024 Job Matching",
    "Rabobank - Fulltime Compensation Levels (as of April 30).Currency",
    "Rabobank - Fulltime Compensation Levels (as of April 30).12 Month Salary",
    "Rabobank - Fulltime Compensation Levels (as of April 30).Holiday Pay",
    "Rabobank - Fulltime Compensation Levels (as of April 30).13th Month",
    "Rabobank - Fulltime Compensation Level

In [28]:
extracted_cols_json = json.loads(competitor_2_columns)
print('type:', type(extracted_cols_json))
print('# cols:', len(extracted_cols_json["headers"]))
print('EXTRACTED COLUMNS:')
print(extracted_cols_json["headers"])
print('type:', type(extracted_cols_json["headers"]))

type: <class 'dict'>
# cols: 51
EXTRACTED COLUMNS:
['Unnamed_0', 'Worker ID', 'Group Entity', 'Board level', 'Board level -1', 'Board level -2', 'Board level -3', 'Supervisory Org', 'Job Code', 'Job Profile Name', 'Position Title', 'Job Family', 'Area of Expertise', 'Gender', 'Date of Birth', 'Age', 'Continuous Service Date', 'Comp Grade', 'Country', 'State', 'Default Working Hours', 'FTE Factor', '2024 Job Matching.KFH', '2024 Job Matching.McLagan', '2024 Job Matching.WTW', '2024 Job Matching.Discipline', '2024 Job Matching', 'Rabobank - Fulltime Compensation Levels (as of April 30).Currency', 'Rabobank - Fulltime Compensation Levels (as of April 30).12 Month Salary', 'Rabobank - Fulltime Compensation Levels (as of April 30).Holiday Pay', 'Rabobank - Fulltime Compensation Levels (as of April 30).13th Month', 'Rabobank - Fulltime Compensation Levels (as of April 30).Salary (based on full time equivalent for those who work part-time; for employees with FTE above 1, amount recalculated t